### Complex Schema Validation with Avro
**Description**: Implement a solution in Python to validate records against a complex nested Avro schema.

Eg., Complex schema ( nested_schema.avsc ):

**Steps**:
1. Load schema
2. Example data to validate
3. Validate against schema
4. Read back to check

In [ ]:
# write your code from here

In [5]:
import json
import avro.schema
import avro.io
import io

# Step 1: Define the schema as a JSON string
schema_json = """
{
  "type": "record",
  "name": "User",
  "fields": [
    {
      "name": "name",
      "type": "string"
    },
    {
      "name": "age",
      "type": "int"
    },
    {
      "name": "address",
      "type": {
        "type": "record",
        "name": "Address",
        "fields": [
          { "name": "street", "type": "string" },
          { "name": "city", "type": "string" },
          { "name": "zipcode", "type": "string" }
        ]
      }
    },
    {
      "name": "phones",
      "type": {
        "type": "array",
        "items": {
          "type": "record",
          "name": "PhoneNumber",
          "fields": [
            { "name": "type", "type": "string" },
            { "name": "number", "type": "string" }
          ]
        }
      }
    }
  ]
}
"""

# Parse the schema
schema = avro.schema.parse(schema_json)

# Step 2: Example data to validate
record = {
    "name": "Alice",
    "age": 30,
    "address": {
        "street": "123 Main St",
        "city": "Wonderland",
        "zipcode": "12345"
    },
    "phones": [
        {"type": "mobile", "number": "123-456-7890"},
        {"type": "home", "number": "098-765-4321"}
    ]
}

# Step 3: Validate and serialize
def validate_and_serialize(data, schema):
    try:
        buffer = io.BytesIO()
        encoder = avro.io.BinaryEncoder(buffer)
        writer = avro.io.DatumWriter(schema)
        writer.write(data, encoder)
        return buffer.getvalue()  # serialized bytes
    except Exception as e:
        print("Validation failed:", e)
        return None

# Step 4: Deserialize and read back
def deserialize(serialized_data, schema):
    buffer = io.BytesIO(serialized_data)
    decoder = avro.io.BinaryDecoder(buffer)
    reader = avro.io.DatumReader(schema)
    return reader.read(decoder)

# Run the full flow
serialized = validate_and_serialize(record, schema)
if serialized:
    print("✅ Record is valid and serialized.")
    restored = deserialize(serialized, schema)
    print("🔁 Read back record:\n", json.dumps(restored, indent=2))
else:
    print("❌ Record validation failed.")


✅ Record is valid and serialized.
🔁 Read back record:
 {
  "name": "Alice",
  "age": 30,
  "address": {
    "street": "123 Main St",
    "city": "Wonderland",
    "zipcode": "12345"
  },
  "phones": [
    {
      "type": "mobile",
      "number": "123-456-7890"
    },
    {
      "type": "home",
      "number": "098-765-4321"
    }
  ]
}
